# Tarea 1 Máquinas de Aprendizaje

## Ejercicio 1

a) Construcción del dataframe. La primera columna del dataframe original es redundante para la indexación, mientras que la columna $Train$ nos permite identificar cuales ejemplos serán parte del training set ($Train = T$) y del testing set ($Train = F$). Para explicitar qué ejemplos son del testing set se invierten los valores de verdad de dicha columna. Finalmente la columna ya utilizada se descarta para quedarnos con las columnas de predictores.

In [24]:
import numpy as np
import pandas as pd

url = 'http://statweb.stanford.edu/~tibs/ElemStatLearn/datasets/prostate.data'
df = pd.read_csv(url, sep='\t', header=0)
# Remover columna con indices redundantes
df = df.drop('Unnamed: 0', axis=1)
# Obtener columna con la etiqueta Train y reemplazar valores booleanos. Estos ejemplos seran de entrenamiento
istrain_str = df['train']
istrain = np.asarray([True if s == 'T' else False for s in istrain_str])
# Listar como testing el resto de valores falsos de la columna anterior
istest = np.logical_not(istrain)
# Una vez procesado los datos, eliminar la columna train para almacenar los predictores relevantes
df = df.drop('train', axis=1)

b) Descripción del dataset

In [25]:
df.shape
df.info()
df.describe()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 97 entries, 0 to 96
Data columns (total 9 columns):
lcavol     97 non-null float64
lweight    97 non-null float64
age        97 non-null int64
lbph       97 non-null float64
svi        97 non-null int64
lcp        97 non-null float64
gleason    97 non-null int64
pgg45      97 non-null int64
lpsa       97 non-null float64
dtypes: float64(5), int64(4)
memory usage: 7.6 KB


,lcavol,lweight,age,lbph,svi,lcp,gleason,pgg45,lpsa
count,97.000000,97.000000,97.000000,97.000000,97.000000,97.000000,97.000000,97.000000,97.000000
mean,1.350010,3.628943,63.865979,0.100356,0.216495,-0.179366,6.752577,24.381443,2.478387
std,1.178625,0.428411,7.445117,1.450807,0.413995,1.398250,0.722134,28.204035,1.154329
min,-1.347074,2.374906,41.000000,-1.386294,0.000000,-1.386294,6.000000,0.000000,-0.430783
25%,0.512824,3.375880,60.000000,-1.386294,0.000000,-1.386294,6.000000,0.000000,1.731656
50%,1.446919,3.623007,65.000000,0.300105,0.000000,-0.798508,7.000000,15.000000,2.591516
75%,2.127041,3.876396,68.000000,1.558145,0.000000,1.178655,7.000000,40.000000,3.056357
max,3.821004,4.780383,79.000000,2.326302,1.000000,2.904165,9.000000,100.000000,5.582932
